The following code Installs the necessary dependencies and mounts Google Drive.

In [ ]:
# Install Libs
!pip install mediapipe
!wget -O face_landmarker_v2_with_blendshapes.task -q https://storage.googleapis.com/mediapipe-models/face_landmarker/face_landmarker/float16/1/face_landmarker.task
!pip install mlxtend

# Mount Drive.
from google.colab import drive
drive.mount('/content/gdrive')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.6/33.6 MB 35.6 MB/s eta 0:00:00
Mounted at /content/gdrive


Code to Visualise the Image using Mediapipe and a Bar graph to view the probabilities of the facial features.

In [ ]:
# Visualisation Tools

from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

def draw_landmarks_on_image(rgb_image, detection_result):
  face_landmarks_list = detection_result.face_landmarks
  annotated_image = np.copy(rgb_image)

  # Loop through the detected faces to visualize.
  for idx in range(len(face_landmarks_list)):
    face_landmarks = face_landmarks_list[idx]

    # Draw the face landmarks.
    face_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    face_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in face_landmarks
    ])

    solutions.drawing_utils.draw_landmarks(
        image=annotated_image,
        landmark_list=face_landmarks_proto,
        connections=mp.solutions.face_mesh.FACEMESH_TESSELATION,
        landmark_drawing_spec=None,
        connection_drawing_spec=mp.solutions.drawing_styles
        .get_default_face_mesh_tesselation_style())
    solutions.drawing_utils.draw_landmarks(
        image=annotated_image,
        landmark_list=face_landmarks_proto,
        connections=mp.solutions.face_mesh.FACEMESH_CONTOURS,
        landmark_drawing_spec=None,
        connection_drawing_spec=mp.solutions.drawing_styles
        .get_default_face_mesh_contours_style())
    solutions.drawing_utils.draw_landmarks(
        image=annotated_image,
        landmark_list=face_landmarks_proto,
        connections=mp.solutions.face_mesh.FACEMESH_IRISES,
          landmark_drawing_spec=None,
          connection_drawing_spec=mp.solutions.drawing_styles
          .get_default_face_mesh_iris_connections_style())

  return annotated_image

def plot_face_blendshapes_bar_graph(face_blendshapes):
  # Extract the face blendshapes category names and scores.
  face_blendshapes_names = [face_blendshapes_category.category_name for face_blendshapes_category in face_blendshapes]
  face_blendshapes_scores = [face_blendshapes_category.score for face_blendshapes_category in face_blendshapes]
  # The blendshapes are ordered in decreasing score value.
  face_blendshapes_ranks = range(len(face_blendshapes_names))

  fig, ax = plt.subplots(figsize=(12, 12))
  bar = ax.barh(face_blendshapes_ranks, face_blendshapes_scores, label=[str(x) for x in face_blendshapes_ranks])
  ax.set_yticks(face_blendshapes_ranks, face_blendshapes_names)
  ax.invert_yaxis()

  # Label each bar with values
  for score, patch in zip(face_blendshapes_scores, bar.patches):
    plt.text(patch.get_x() + patch.get_width(), patch.get_y(), f"{score:.4f}", va="top")

  ax.set_xlabel('Score')
  ax.set_title("Face Blendshapes")
  plt.tight_layout()
  plt.show()

Code to Show the Image.

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
def show_image(image_path):
  img = cv2.imread(image_path)
  cv2_imshow(img)

# show_image("/content/gdrive/MyDrive/images/1/Anger.jpg")

Main Functions to Extract Facial Features and Define Apriori

In [ ]:
from typing import Any
import mediapipe as mp
from mediapipe.tasks import python
from collections import defaultdict, Counter
from itertools import combinations
from mediapipe.tasks.python import vision

# This Function passes an image into Mediapipe and returns the results of the image.
# image_path - The path of the image.
# show_visuals - Outputs the annotations and the bar graph of the image result.

# Returns a Mediapipe Result object.
def get_features(image_path: str, show_visuals: bool = False) -> Any:
  base_options = python.BaseOptions(model_asset_path='face_landmarker_v2_with_blendshapes.task')
  options = vision.FaceLandmarkerOptions(base_options=base_options,
                                        output_face_blendshapes=True,
                                        output_facial_transformation_matrixes=True,
                                        num_faces=1)
  detector = vision.FaceLandmarker.create_from_options(options)
  image = mp.Image.create_from_file(image_path)
  detection_result = detector.detect(image)

  if show_visuals:
    annotated_image = draw_landmarks_on_image(image.numpy_view(), detection_result)
    show_image(image_path)
    cv2_imshow(cv2.cvtColor(annotated_image, cv2.COLOR_RGB2BGR))

  return detection_result

# This function obtains the major features of a particular Image.
# result - The result obtained by the MediaPipe for an image.
# threshold - The probability threshold for the feature to be accepted.
# add_label - Whether to add the label to the list of features or not.
# label - The label to be added.

# Returns a list of strings containing the major features.
def get_major_features(result, threshold: float, add_label: bool = False, label: str = None) -> list[str]:
  features_list = result.face_blendshapes[0]
  low = min(features_list, key=lambda x: x.score).score
  high = max(features_list, key=lambda x: x.score).score
  n = high - low
  out = [feature.category_name for feature in features_list if (feature.score - low) / n >= threshold]
  if add_label:
    out.append(label)
  return out

# This function obtains the features of all the images in the list.
# images_list - The list of images whose features are to be obtained.
# threshold - The probability threshold for the feature to be accepted.
# add_label - Whether to add the label to the list of features or not.
# label - The label to be added.

# List of List of features for each image.
def get_dataset_features(images_list: list[str], threshold: float, add_label: bool = False, label: str = None) -> list[list[str]]:
  out = []
  p, np = 0, 0
  for i, image in enumerate(images_list):
    print(f"{image}")
    try:
      result = get_features(image)
      out.append(get_major_features(result, threshold, add_label, label))
      p += 1
    except:
      np += 1
  return out

# Define Apriori
# data - The list of features
# min_support - The minimum percentage to be considered frequent.
# min_confidence - The minimum confidence for a rule to be valid.
# give_all_rules - Returns rules from all iterations if true.

# Return Itemset (list) and the rules (dict)
def apriori(data: list[list[str]], min_support: float, min_confidence: float, give_all_rules: bool = False) -> tuple:
  rules = {}

  # Get Inital Items
  init = []
  for i in data:
      for q in i:
          if(q not in init):
              init.append(q)
  init = sorted(init)

  # Get Frequent Inits.
  c = Counter()
  for i in init:
      for d in data:
          if(i in d):
              c[i]+=1

  s = int(min_support*len(init))
  l = Counter()
  for i in c:
      if(c[i] >= s):
          l[frozenset([i])]+=c[i]

  pl = l
  pos = 1
  for count in range (2,1000):
      nc = set()
      temp = list(l)
      for i in range(0,len(temp)):
          for j in range(i+1,len(temp)):
              t = temp[i].union(temp[j])
              if(len(t) == count):
                  nc.add(temp[i].union(temp[j]))
      nc = list(nc)

      c = Counter()
      for i in nc:
          c[i] = 0
          for q in data:
              temp = set(q)
              if(i.issubset(temp)):
                  c[i]+=1

      l = Counter()
      for i in c:
          if(c[i] >= s):
              l[i]+=c[i]

      if(len(l) == 0):
          break
      if give_all_rules:
        rules.update(get_rules(get_association_rules(l, data), min_confidence))
      pl = l
      pos = count

  for l in pl:
      c = [frozenset(q) for q in combinations(l,len(l)-1)]
      mmax = 0
      for a in c:
          b = l-a
          ab = l
          sab = 0
          sa = 0
          sb = 0
          for q in data:
              temp = set(q)
              if(a.issubset(temp)):
                  sa+=1
              if(b.issubset(temp)):
                  sb+=1
              if(ab.issubset(temp)):
                  sab+=1
          temp = sab/sa*100
          if(temp > mmax):
              mmax = temp
          temp = sab/sb*100
          if(temp > mmax):
              mmax = temp

      curr = 1
      for a in c:
          b = l-a
          ab = l
          sab = 0
          sa = 0
          sb = 0
          for q in data:
              temp = set(q)
              if(a.issubset(temp)):
                  sa+=1
              if(b.issubset(temp)):
                  sb+=1
              if(ab.issubset(temp)):
                  sab+=1
          temp = sab/sa*100
          curr += 1
          temp = sab/sb*100
          curr += 1

  itemset = []
  for key in pl.keys():
    itemset.append(key)
  if not give_all_rules:
    rules = get_rules(get_association_rules(pl, data), min_confidence)
  return itemset, rules

def get_association_rules(l_set, data):
  out = ""
  for l in l_set:
      c = [frozenset(q) for q in combinations(l,len(l)-1)]
      mmax = 0
      for a in c:
          b = l-a
          ab = l
          sab = 0
          sa = 0
          sb = 0
          for q in data:
              temp = set(q)
              if(a.issubset(temp)):
                  sa+=1
              if(b.issubset(temp)):
                  sb+=1
              if(ab.issubset(temp)):
                  sab+=1
          temp = sab/sa*100
          if(temp > mmax):
              mmax = temp
          temp = sab/sb*100
          if(temp > mmax):
              mmax = temp
          out += str(list(a))+" -> "+str(list(b))+" = "+str(sab/sa*100)+"%\n"
          out += str(list(b))+" -> "+str(list(a))+" = "+str(sab/sb*100)+"%\n"
  return out

def breakdown_rule(rule, threshold):
  rule.strip()
  if rule == "":
    return None, None
  left, right = rule.split("->")
  right, confidence = right.split("=")
  confidence = float(confidence.strip()[:-1])
  if confidence < (threshold * 100):
    return None, None
  left = left.strip()[1:-1]
  left = [x.strip()[1:-1] for x in left.split(",")]
  right = right.strip()[1:-1]
  right = [x.strip()[1:-1] for x in right.split(",")]
  return left, right
def get_rules(text, confidence):
  rules = defaultdict(set)
  rules_text = text.split("\n")
  for rule in rules_text:
    left, right = breakdown_rule(rule, confidence)
    if left is None or right is None:
      continue
    rules[frozenset(left)].update(right)
  return rules

def clean_output(itemset, rules):
  for i, item in enumerate(itemset):
    itemset[i] = set(item)
  rule_list = []
  for k, v in rules.items():
    rule_list.append((set(k), set(v)))
  rule_list.sort(key = lambda x: (len(x[0]), len(x[1])))
  return itemset, rule_list

In [ ]:
for feature in get_features("/content/gdrive/MyDrive/DFEW/Surprise/16361.jpg").face_blendshapes[0]:
  print(feature)

Category(index=0, score=7.581438126180728e-07, display_name='', category_name='_neutral')
Category(index=1, score=0.008888531476259232, display_name='', category_name='browDownLeft')
Category(index=2, score=0.014593943953514099, display_name='', category_name='browDownRight')
Category(index=3, score=0.13123111426830292, display_name='', category_name='browInnerUp')
Category(index=4, score=0.10721280425786972, display_name='', category_name='browOuterUpLeft')
Category(index=5, score=0.05887880176305771, display_name='', category_name='browOuterUpRight')
Category(index=6, score=1.6084772141766734e-05, display_name='', category_name='cheekPuff')
Category(index=7, score=1.1607834267124417e-06, display_name='', category_name='cheekSquintLeft')
Category(index=8, score=4.416201022650057e-07, display_name='', category_name='cheekSquintRight')
Category(index=9, score=0.04112816974520683, display_name='', category_name='eyeBlinkLeft')
Category(index=10, score=0.015555859543383121, display_name='

In [ ]:
import os
labels = ["Angry", "Disgust", "Fear", "Happy", "Neutral", "Sad", "Surprise"]
count = 0
can = 0
with open("/content/gdrive/MyDrive/DFEW/features_data.csv", "w") as file:
  for label in labels:
    path = f"/content/gdrive/MyDrive/DFEW/{label}"
    images = [f"{path}/{image}" for image in os.listdir(path)]
    for i, image in enumerate(images):
      if i % 50 == 0:
        print(f"{label} - Images Complete - {i} / {len(images)}")
      try:
        features = get_features(image).face_blendshapes[0]
        feature_data = []
        for feature in features:
          feature_data.append(feature.category_name)
          feature_data.append(str(feature.score))
        file.write(f"{image},{label},{','.join(feature_data)}\n")
        count += 1
      except IndexError:
        can += 1
        continue
  print(f"Processed - {label}")
print(f"Processed - {count}\nCancelled - {can}")

Angry - Images Complete - 0 / 2173
Angry - Images Complete - 50 / 2173
Angry - Images Complete - 100 / 2173
Angry - Images Complete - 150 / 2173
Angry - Images Complete - 200 / 2173
Angry - Images Complete - 250 / 2173
Angry - Images Complete - 300 / 2173
Angry - Images Complete - 350 / 2173
Angry - Images Complete - 400 / 2173
Angry - Images Complete - 450 / 2173
Angry - Images Complete - 500 / 2173
Angry - Images Complete - 550 / 2173
Angry - Images Complete - 600 / 2173
Angry - Images Complete - 650 / 2173
Angry - Images Complete - 700 / 2173
Angry - Images Complete - 750 / 2173
Angry - Images Complete - 800 / 2173
Angry - Images Complete - 850 / 2173
Angry - Images Complete - 900 / 2173
Angry - Images Complete - 950 / 2173
Angry - Images Complete - 1000 / 2173
Angry - Images Complete - 1050 / 2173
Angry - Images Complete - 1100 / 2173
Angry - Images Complete - 1150 / 2173
Angry - Images Complete - 1200 / 2173
Angry - Images Complete - 1250 / 2173
Angry - Images Complete - 1300 / 21

Code to generate the CSV files for features.

In [ ]:
import os
labels = ["Angry", "Disgust", "Fear", "Happy", "Neutral", "Sad", "Surprise"]
count = 0
can = 0
with open("/content/gdrive/MyDrive/DFEW/features_data_0.4.csv", "w") as file:
  for label in labels:
    path = f"/content/gdrive/MyDrive/DFEW/{label}"
    images = [f"{path}/{image}" for image in os.listdir(path)]
    for i, image in enumerate(images):
      if i % 50 == 0:
        print(f"{label} - Images Complete - {i} / {len(images)}")
      try:
        feature = get_major_features(get_features(image), 0.4, False)
        file.write(f"{image},{label},{','.join(feature)}\n")
        count += 1
      except IndexError:
        can += 1
        continue
  print(f"Processed - {label}")
print(f"Processed - {count}\nCancelled - {can}")

Angry - Images Complete - 0 / 2173
Angry - Images Complete - 50 / 2173
Angry - Images Complete - 100 / 2173
Angry - Images Complete - 150 / 2173
Angry - Images Complete - 200 / 2173
Angry - Images Complete - 250 / 2173
Angry - Images Complete - 300 / 2173
Angry - Images Complete - 350 / 2173
Angry - Images Complete - 400 / 2173
Angry - Images Complete - 450 / 2173
Angry - Images Complete - 500 / 2173
Angry - Images Complete - 550 / 2173
Angry - Images Complete - 600 / 2173
Angry - Images Complete - 650 / 2173
Angry - Images Complete - 700 / 2173
Angry - Images Complete - 750 / 2173
Angry - Images Complete - 800 / 2173
Angry - Images Complete - 850 / 2173
Angry - Images Complete - 900 / 2173
Angry - Images Complete - 950 / 2173
Angry - Images Complete - 1000 / 2173
Angry - Images Complete - 1050 / 2173
Angry - Images Complete - 1100 / 2173
Angry - Images Complete - 1150 / 2173
Angry - Images Complete - 1200 / 2173
Angry - Images Complete - 1250 / 2173
Angry - Images Complete - 1300 / 21

Old Code


In [ ]:
import os

FEATURE_ACCEPT_THRESHOLD = 0.6
APRIORI_SUPPORT_THRESHOLD = 0.3
APRIORI_CONFIDENCE_THRESHOLD = 0
labels = ["Angry", "Disgust", "Fear", "Happy", "Neutral", "Sad", "Surprise"]

class Emotion:
  def __init__(self, itemset, rules, label):
    self.itemset = itemset
    self.rules = rules
    self.label = label

emotion_outputs = {}
for label in ["Happy"]:
  path = f"/content/gdrive/MyDrive/DFEW/{label}"
  features = get_dataset_features([f"{path}/{image}" for image in os.listdir(path)], FEATURE_ACCEPT_THRESHOLD, True, label)
  itemset, rules = a2(features, APRIORI_SUPPORT_THRESHOLD, APRIORI_CONFIDENCE_THRESHOLD)
  itemset, rules = clean_output(itemset, rules)
  emotion_outputs[label] = Emotion(itemset, rules, label)

# # All Labels
# features = []
# for label in labels:
#   features.extend(get_dataset_features([f"/content/gdrive/MyDrive/images/{i}/{label}.jpg" for i in range(19)], FEATURE_ACCEPT_THRESHOLD, True, label))
# itemset, rules = a2(features, APRIORI_SUPPORT_THRESHOLD, APRIORI_CONFIDENCE_THRESHOLD)
# print(rules)
# for item in set(itemset):
#   print(item)
# for k, v in rules:
#   print(f"{k} -> {v}")

# # Individual Labels
# emotion_outputs = {}
# for label in labels:
  # features = get_dataset_features([f"/content/gdrive/MyDrive/DFEW/{label}.jpg" for i in range(19)], FEATURE_ACCEPT_THRESHOLD, True, label)
  # itemset, rules = a2(features, APRIORI_SUPPORT_THRESHOLD, APRIORI_CONFIDENCE_THRESHOLD)
  # itemset, rules = clean_output(itemset, rules)
  # emotion_outputs[label] = Emotion(itemset, rules, label)

# for label in labels:
#   print(f"Emotion - {label}")
#   print("Itemsets")
#   for itemset in emotion_outputs[label].itemset:
#     print(itemset)
#   print()
#   for k, v in emotion_outputs[label].rules:
#     print(f"{k} -> {v}")
#   print("----------------------------------------------------------------")

/content/gdrive/MyDrive/DFEW/Happy/9780.jpg
/content/gdrive/MyDrive/DFEW/Happy/10401.jpg
/content/gdrive/MyDrive/DFEW/Happy/10521.jpg
/content/gdrive/MyDrive/DFEW/Happy/9052.jpg
/content/gdrive/MyDrive/DFEW/Happy/10495.jpg
/content/gdrive/MyDrive/DFEW/Happy/8765.jpg
/content/gdrive/MyDrive/DFEW/Happy/9258.jpg
/content/gdrive/MyDrive/DFEW/Happy/9771.jpg
/content/gdrive/MyDrive/DFEW/Happy/9866.jpg
/content/gdrive/MyDrive/DFEW/Happy/11134.jpg
/content/gdrive/MyDrive/DFEW/Happy/10483.jpg
/content/gdrive/MyDrive/DFEW/Happy/10098.jpg
/content/gdrive/MyDrive/DFEW/Happy/10384.jpg
/content/gdrive/MyDrive/DFEW/Happy/10127.jpg
/content/gdrive/MyDrive/DFEW/Happy/11179.jpg
/content/gdrive/MyDrive/DFEW/Happy/10347.jpg
/content/gdrive/MyDrive/DFEW/Happy/9744.jpg
/content/gdrive/MyDrive/DFEW/Happy/11159.jpg
/content/gdrive/MyDrive/DFEW/Happy/10991.jpg
/content/gdrive/MyDrive/DFEW/Happy/10409.jpg
/content/gdrive/MyDrive/DFEW/Happy/10712.jpg
/content/gdrive/MyDrive/DFEW/Happy/10852.jpg
/content/gdrive/M

In [ ]:
from collections import defaultdict
import random

APRIORI_SUPPORT_THRESHOLD = 0.3
APRIORI_CONFIDENCE_THRESHOLD = 1
labels = ["Angry", "Disgust", "Fear", "Happy", "Neutral", "Sad", "Surprise"]

SUPPORT_THRESHOLD = 0.6
# Get Data
data = defaultdict(dict)
with open(f"/content/gdrive/MyDrive/DFEW/features_data_{SUPPORT_THRESHOLD}.csv") as file:
  for line in file:
    values = list(map(lambda x: x.strip(), line.split(",")))
    data[values[1]][values[0]] = values[2:]


In [ ]:
result = get_features("/content/gdrive/MyDrive/images/18/Happy.jpg")
l = []
for cat in result.face_blendshapes[0]:
  l.append(cat.category_name)
print(len(l))

52


## Approach 1
- No Labels in the feature set during training.
- Rules are generated per emotion.
- Rules are generated only from the last Ruleset.

### Algorithm
1. Separate images as train and test.
2. Generate Rules and Itemsets for every emotion using the train data for that particular emotion.
3. To test the image, every image's feature set is passed throught the rules of the emotion to be tested to get an extended feature set. Finally, if the itemset is a subset of the extended feature set, then the emotion is present.

## Output

Angry-Angry -> Correct: 14, Incorrect: 579 \\
Angry-Disgust -> Correct: 36, Incorrect: 6 \\
Angry-Fear -> Correct: 229, Incorrect: 7 \\
Angry-Happy -> Correct: 669, Incorrect: 21 \\
Angry-Neutral -> Correct: 718, Incorrect: 26 \\
Angry-Sad -> Correct: 472, Incorrect: 38 \\
Angry-Surprise -> Correct: 419, Incorrect: 3 \\
Disgust-Angry -> Correct: 574, Incorrect: 19 \\
Disgust-Disgust -> Correct: 7, Incorrect: 35 \\
Disgust-Fear -> Correct: 230, Incorrect: 6 \\
Disgust-Happy -> Correct: 659, Incorrect: 31 \\
Disgust-Neutral -> Correct: 711, Incorrect: 33 \\
Disgust-Sad -> Correct: 465, Incorrect: 45 \\
Disgust-Surprise -> Correct: 418, Incorrect: 4 \\
Fear-Angry -> Correct: 579, Incorrect: 14 \\
Fear-Disgust -> Correct: 41, Incorrect: 1 \\
Fear-Fear -> Correct: 6, Incorrect: 230 \\
Fear-Happy -> Correct: 659, Incorrect: 31 \\
Fear-Neutral -> Correct: 716, Incorrect: 28 \\
Fear-Sad -> Correct: 501, Incorrect: 9 \\
Fear-Surprise -> Correct: 404, Incorrect: 18 \\
Happy-Angry -> Correct: 587, Incorrect: 6 \\
Happy-Disgust -> Correct: 42, Incorrect: 0 \\
Happy-Fear -> Correct: 236, Incorrect: 0 \\
Happy-Happy -> Correct: 7, Incorrect: 683 \\
Happy-Neutral -> Correct: 744, Incorrect: 0 \\
Happy-Sad -> Correct: 504, Incorrect: 6 \\
Happy-Surprise -> Correct: 422, Incorrect: 0 \\
Neutral-Angry -> Correct: 569, Incorrect: 24 \\
Neutral-Disgust -> Correct: 36, Incorrect: 6 \\
Neutral-Fear -> Correct: 227, Incorrect: 9 \\
Neutral-Happy -> Correct: 655, Incorrect: 35 \\
Neutral-Neutral -> Correct: 42, Incorrect: 702 \\
Neutral-Sad -> Correct: 465, Incorrect: 45 \\
Neutral-Surprise -> Correct: 414, Incorrect: 8 \\
Sad-Angry -> Correct: 578, Incorrect: 15 \\
Sad-Disgust -> Correct: 37, Incorrect: 5 \\
Sad-Fear -> Correct: 229, Incorrect: 7 \\
Sad-Happy -> Correct: 651, Incorrect: 39 \\
Sad-Neutral -> Correct: 704, Incorrect: 40 \\
Sad-Sad -> Correct: 48, Incorrect: 462 \\
Sad-Surprise -> Correct: 418, Incorrect: 4 \\
Surprise-Angry -> Correct: 587, Incorrect: 6 \\
Surprise-Disgust -> Correct: 42, Incorrect: 0 \\
Surprise-Fear -> Correct: 227, Incorrect: 9 \\
Surprise-Happy -> Correct: 671, Incorrect: 19 \\
Surprise-Neutral -> Correct: 719, Incorrect: 25 \\
Surprise-Sad -> Correct: 505, Incorrect: 5 \\
Surprise-Surprise -> Correct: 22, Incorrect: 400 \\

In [ ]:
import pickle
class Emotion:
  def __init__(self, itemset, rules, label):
    self.itemset = itemset
    self.rules = rules
    self.label = label

emotion_outputs = {}
train = {}
test = {}
for label in labels:
  train_keys = random.sample([*data[label].keys()], int(len(data[label]) * 0.7))
  test_keys = list(data[label].keys() - set(train_keys))
  train[label] = train_keys
  test[label] = test_keys

  features = [data[label][image] for image in train[label]]
  itemset, rules = apriori(features, APRIORI_SUPPORT_THRESHOLD, APRIORI_CONFIDENCE_THRESHOLD)
  itemset, rules = clean_output(itemset, rules)
  emotion_outputs[label] = Emotion(itemset, rules, label)
  print(f"{label} Done")
with open("rules_1_0.6_0.3_1.prsg", "wb") as file:
  pickle.dump(emotion_outputs, file)

with open("rules_1_0.6_0.3_1.prsg", "rb") as file:
  a = pickle.load(file)

# out = defaultdict(dict)
# for label in labels:
#   for test_label in labels:
#     c, ic = 0, 0
#     for image in test[test_label]:
#       features = set(data[test_label][image])
#       for key, value in emotion_outputs[label].rules:
#         if key.issubset(features):
#           features.update(set(value))
#       for item in emotion_outputs[label].itemset:
#         if set(item).issubset(features):
#           c += 1
#           break
#       else:
#         ic += 1
#     if label == test_label:
#       out[label][test_label] = (c, ic)
#     else:
#       out[label][test_label] = (ic, c)
# for a in labels:
#   for b in labels:
#     print(f"{a}-{b} -> Correct: {out[a][b][0]}, Incorrect: {out[a][b][1]}")

Angry Done
Disgust Done
Fear Done
Happy Done
Neutral Done
Sad Done
Surprise Done
[({'eyeLookDownRight', 'eyeBlinkLeft', 'mouthSmileLeft', 'eyeLookDownLeft', 'mouthSmileRight', 'eyeSquintRight', 'eyeBlinkRight'}, {'eyeSquintLeft'}), ({'eyeLookDownRight', 'eyeBlinkLeft', 'eyeLookDownLeft', 'eyeSquintLeft', 'mouthSmileRight', 'eyeSquintRight', 'eyeBlinkRight'}, {'mouthSmileLeft'}), ({'eyeLookDownRight', 'eyeBlinkLeft', 'mouthSmileLeft', 'eyeSquintLeft', 'mouthSmileRight', 'eyeSquintRight', 'eyeBlinkRight'}, {'eyeLookDownLeft'}), ({'eyeBlinkLeft', 'mouthSmileLeft', 'eyeLookDownLeft', 'eyeSquintLeft', 'mouthSmileRight', 'eyeSquintRight', 'eyeBlinkRight'}, {'eyeLookDownRight'})]
[({'eyeLookDownRight', 'eyeLookDownLeft', 'mouthSmileRight', 'eyeSquintRight', 'eyeBlinkLeft', 'mouthSmileLeft', 'eyeBlinkRight'}, {'eyeSquintLeft'}), ({'eyeLookDownRight', 'eyeLookDownLeft', 'mouthSmileRight', 'eyeSquintRight', 'eyeBlinkLeft', 'eyeSquintLeft', 'eyeBlinkRight'}, {'mouthSmileLeft'}), ({'eyeLookDownRig

## Approach 2
- Labels present in the feature set during training.
- Rules are generated per emotion.
- Rules are generated only from the last Ruleset.

### Algorithm
1. Separate images as train and test.
2. Generate Rules and Itemsets for every emotion using the train data for that particular emotion.
3. To test the image, every image's feature set is passed through the rules of the emotion to be tested (which does not contain the emotion in the antecedent) to get an extended feature set. Finally, if the extended feature set contains the emotion, then the emotion is present.

## Output

Angry-Angry -> Correct: 10, Incorrect: 583 \\
Angry-Disgust -> Correct: 37, Incorrect: 5 \\
Angry-Fear -> Correct: 235, Incorrect: 1 \\
Angry-Happy -> Correct: 673, Incorrect: 17 \\
Angry-Neutral -> Correct: 721, Incorrect: 23 \\
Angry-Sad -> Correct: 481, Incorrect: 29 \\
Angry-Surprise -> Correct: 422, Incorrect: 0 \\
Disgust-Angry -> Correct: 564, Incorrect: 29 \\
Disgust-Disgust -> Correct: 6, Incorrect: 36 \\
Disgust-Fear -> Correct: 224, Incorrect: 12 \\
Disgust-Happy -> Correct: 638, Incorrect: 52 \\
Disgust-Neutral -> Correct: 701, Incorrect: 43 \\
Disgust-Sad -> Correct: 456, Incorrect: 54 \\
Disgust-Surprise -> Correct: 411, Incorrect: 11 \\
Fear-Angry -> Correct: 569, Incorrect: 24 \\
Fear-Disgust -> Correct: 32, Incorrect: 10 \\
Fear-Fear -> Correct: 10, Incorrect: 226 \\
Fear-Happy -> Correct: 648, Incorrect: 42 \\
Fear-Neutral -> Correct: 687, Incorrect: 57 \\
Fear-Sad -> Correct: 456, Incorrect: 54 \\
Fear-Surprise -> Correct: 407, Incorrect: 15 \\
Happy-Angry -> Correct: 592, Incorrect: 1 \\
Happy-Disgust -> Correct: 42, Incorrect: 0 \\
Happy-Fear -> Correct: 236, Incorrect: 0 \\
Happy-Happy -> Correct: 4, Incorrect: 686 \\
Happy-Neutral -> Correct: 744, Incorrect: 0 \\
Happy-Sad -> Correct: 507, Incorrect: 3 \\
Happy-Surprise -> Correct: 422, Incorrect: 0 \\
Neutral-Angry -> Correct: 572, Incorrect: 21 \\
Neutral-Disgust -> Correct: 37, Incorrect: 5 \\
Neutral-Fear -> Correct: 229, Incorrect: 7 \\
Neutral-Happy -> Correct: 651, Incorrect: 39 \\
Neutral-Neutral -> Correct: 42, Incorrect: 702 \\
Neutral-Sad -> Correct: 464, Incorrect: 46 \\
Neutral-Surprise -> Correct: 417, Incorrect: 5 \\
Sad-Angry -> Correct: 592, Incorrect: 1 \\
Sad-Disgust -> Correct: 41, Incorrect: 1 \\
Sad-Fear -> Correct: 236, Incorrect: 0 \\
Sad-Happy -> Correct: 686, Incorrect: 4 \\
Sad-Neutral -> Correct: 740, Incorrect: 4 \\
Sad-Sad -> Correct: 2, Incorrect: 508 \\
Sad-Surprise -> Correct: 422, Incorrect: 0 \\
Surprise-Angry -> Correct: 584, Incorrect: 9 \\
Surprise-Disgust -> Correct: 40, Incorrect: 2 \\
Surprise-Fear -> Correct: 226, Incorrect: 10 \\
Surprise-Happy -> Correct: 687, Incorrect: 3 \\
Surprise-Neutral -> Correct: 723, Incorrect: 21 \\
Surprise-Sad -> Correct: 502, Incorrect: 8 \\
Surprise-Surprise -> Correct: 18, Incorrect: 404 \\

In [ ]:
class Emotion:
  def __init__(self, itemset, rules, label):
    self.itemset = itemset
    self.rules = rules
    self.label = label

emotion_outputs = {}
train = {}
test = {}
for label in labels:
  train_keys = random.sample([*data[label].keys()], int(len(data[label]) * 0.7))
  test_keys = list(data[label].keys() - set(train_keys))
  train[label] = train_keys
  test[label] = test_keys

  features = [data[label][image] + [label] for image in train[label]]
  itemset, rules = a2(features, APRIORI_SUPPORT_THRESHOLD, APRIORI_CONFIDENCE_THRESHOLD)
  itemset, rules = clean_output(itemset, rules)
  emotion_outputs[label] = Emotion(itemset, rules, label)
  print(f"{label} Done")

out = defaultdict(dict)
for label in labels:
  for test_label in labels:
    c, ic = 0, 0
    for image in test[test_label]:
      features = set(data[test_label][image])
      for key, value in emotion_outputs[label].rules:
        if label in key:
          continue
        if key.issubset(features):
          features.update(set(value))
      if label in features:
          c += 1
      else:
        ic += 1
    if label == test_label:
      out[label][test_label] = (c, ic)
    else:
      out[label][test_label] = (ic, c)
for a in labels:
  for b in labels:
    print(f"{a}-{b} -> Correct: {out[a][b][0]}, Incorrect: {out[a][b][1]}")

## Approach 3
- Labels present in the feature set during training.
- Rules are generated for all emotions.
- Rules are generated only from the last Ruleset.

In [ ]:
train = []
test = []
features = []
for label in labels:
  train_keys = random.sample([*data[label].keys()], int(len(data[label]) * 0.7))
  test_keys = list(data[label].keys() - set(train_keys))
  train.extend(train_keys)
  test.extend(test_keys)

  features.extend([data[label][image] + [label] for image in train_keys])

itemset, rules = apriori(features, APRIORI_SUPPORT_THRESHOLD, APRIORI_CONFIDENCE_THRESHOLD)
itemset, rules = clean_output(itemset, rules)

for item in itemset:
  print(item)
print()
for key, value in rules:
  print(f"{set(key)} -> {set(value)}")

{'mouthSmileRight', 'eyeSquintRight', 'eyeLookDownLeft', 'mouthSmileLeft', 'eyeLookDownRight', 'Happy', 'eyeBlinkLeft', 'eyeSquintLeft', 'eyeBlinkRight'}

{'mouthSmileRight', 'eyeLookDownLeft', 'mouthSmileLeft', 'eyeLookDownRight', 'Happy', 'eyeBlinkLeft', 'eyeSquintRight', 'eyeBlinkRight'} -> {'eyeSquintLeft'}
{'mouthSmileRight', 'eyeSquintRight', 'eyeLookDownLeft', 'mouthSmileLeft', 'Happy', 'eyeBlinkLeft', 'eyeSquintLeft', 'eyeBlinkRight'} -> {'eyeLookDownRight'}
{'mouthSmileRight', 'eyeSquintRight', 'eyeLookDownLeft', 'eyeLookDownRight', 'Happy', 'eyeBlinkLeft', 'eyeSquintLeft', 'eyeBlinkRight'} -> {'mouthSmileLeft'}
{'mouthSmileRight', 'eyeSquintRight', 'mouthSmileLeft', 'eyeLookDownRight', 'Happy', 'eyeBlinkLeft', 'eyeSquintLeft', 'eyeBlinkRight'} -> {'eyeLookDownLeft'}


## Approach 4
- No Labels in the feature set during training.
- Rules are generated per emotion.
- Rules are generated only all Rulesets.

## Approach 5
- Labels present in the feature set during training.
- Rules are generated per emotion.
- Rules are generated only all Rulesets.

## Approach 6
- Labels present in the feature set during training.
- Rules are generated for all emotion.
- Rules are generated only all Rulesets.

In [ ]:
for label in labels:
  print(label)
  print()
  for item in emotion_outputs[label].itemset:
    print(item)
  print()
  for key, value in emotion_outputs[label].rules:
    print(f"{set(key)} -> {set(value)}")
  print("-" * 50)

Angry

{'eyeBlinkRight', 'eyeLookDownLeft', 'Angry', 'eyeSquintRight', 'eyeSquintLeft', 'eyeLookDownRight', 'eyeBlinkLeft'}

{'eyeBlinkRight', 'eyeLookDownLeft', 'Angry', 'eyeSquintLeft', 'eyeSquintRight', 'eyeBlinkLeft'} -> {'eyeLookDownRight'}
{'eyeBlinkRight', 'eyeLookDownLeft', 'eyeSquintRight', 'eyeSquintLeft', 'eyeLookDownRight', 'eyeBlinkLeft'} -> {'Angry'}
{'eyeBlinkRight', 'Angry', 'eyeSquintRight', 'eyeSquintLeft', 'eyeLookDownRight', 'eyeBlinkLeft'} -> {'eyeLookDownLeft'}
--------------------------------------------------
Disgust

{'eyeBlinkRight', 'eyeLookDownLeft', 'eyeSquintLeft', 'eyeLookDownRight', 'Disgust', 'eyeBlinkLeft'}
{'eyeBlinkRight', 'eyeLookDownLeft', 'eyeLookDownRight', 'eyeSquintLeft', 'eyeSquintRight', 'Disgust'}
{'eyeLookDownLeft', 'eyeLookDownRight', 'eyeSquintLeft', 'eyeSquintRight', 'Disgust', 'eyeBlinkLeft'}

{'eyeBlinkRight', 'eyeLookDownLeft', 'eyeSquintLeft', 'Disgust', 'eyeBlinkLeft'} -> {'eyeLookDownRight'}
{'eyeBlinkRight', 'eyeLookDownLeft', 'ey

In [ ]:
out = defaultdict(dict)
for label in labels:
  for test_label in labels:
    c, ic = 0, 0
    for image in test[test_label]:
      features = set(data[test_label][image])
      for key, value in emotion_outputs[label].rules:
        if key.issubset(features):
          features.update(set(value))
      for item in emotion_outputs[label].itemset:
        if set(item).issubset(features):
          c += 1
          break
      else:
        ic += 1
    if label == test_label:
      out[label][test_label] = (c, ic)
    else:
      out[label][test_label] = (ic, c)
for a in labels:
  for b in labels:
    print(f"{a}-{b} -> Correct: {out[a][b][0]}, Incorrect: {out[a][b][1]}")

Angry-Angry -> Correct: 14, Incorrect: 579
Angry-Disgust -> Correct: 36, Incorrect: 6
Angry-Fear -> Correct: 229, Incorrect: 7
Angry-Happy -> Correct: 669, Incorrect: 21
Angry-Neutral -> Correct: 718, Incorrect: 26
Angry-Sad -> Correct: 472, Incorrect: 38
Angry-Surprise -> Correct: 419, Incorrect: 3
Disgust-Angry -> Correct: 574, Incorrect: 19
Disgust-Disgust -> Correct: 7, Incorrect: 35
Disgust-Fear -> Correct: 230, Incorrect: 6
Disgust-Happy -> Correct: 659, Incorrect: 31
Disgust-Neutral -> Correct: 711, Incorrect: 33
Disgust-Sad -> Correct: 465, Incorrect: 45
Disgust-Surprise -> Correct: 418, Incorrect: 4
Fear-Angry -> Correct: 579, Incorrect: 14
Fear-Disgust -> Correct: 41, Incorrect: 1
Fear-Fear -> Correct: 6, Incorrect: 230
Fear-Happy -> Correct: 659, Incorrect: 31
Fear-Neutral -> Correct: 716, Incorrect: 28
Fear-Sad -> Correct: 501, Incorrect: 9
Fear-Surprise -> Correct: 404, Incorrect: 18
Happy-Angry -> Correct: 587, Incorrect: 6
Happy-Disgust -> Correct: 42, Incorrect: 0
Happy